# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [74]:
from mxnet import ndarray as nd
import numpy as np
import time

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [75]:
# 1. Construct two matrices A and B with Gaussian random entries of size  4096×4096. 
A = nd.array(nd.random.normal(shape=(4096,4096)))
B = nd.array(nd.random.normal(shape=(4096,4096)))
print("A is ", A)
print("B is ", B)
# 2. Compute C=AB using matrix-matrix operations and report the time.
tic = time.time()
C = nd.dot(A,B)
C.wait_to_read()
print("C is ", C)
print("Time taken to compute C = AB using matrix-matrix operations is ",time.time() - tic)
# 3. Compute C=AB, treating A as a matrix but computing the result for each column 
# of B one at a time. Report the time.
tic = time.time()
for i in range(4096):
    C[:,i] = nd.dot(A, B[:,i])
C.wait_to_read()
print("C is ", C)
print("Time taken to compute C = AB using a series of matrix-vector operations is ",time.time() - tic)
# 4. Compute C=AB, treating A and B as collections of vectors. Report the time.
tic = time.time()
C = nd.zeros((4096,4096))
for i in range(4096):
    A_ = A[:,i].reshape(4096,1)
    B_ = B[i,:].reshape(1,4096)
    C += nd.dot(A_,B_)
C.wait_to_read()
print("C is ", C)
print("Time taken to compute C = AB using a series of matrix-vector operations is ",time.time()- tic)


A is  
[[-0.13966009 -0.38877     0.24898222 ...  0.14593127  0.92136323
   1.1953865 ]
 [-1.1556278   0.63732547  0.89197177 ...  1.2996469   1.0591276
   0.25805908]
 [-0.12369207 -0.89886904 -1.2136172  ... -0.19027245  0.5325477
  -0.9198402 ]
 ...
 [-1.0942916  -0.08565583 -0.04852283 ...  1.490433    1.654896
  -0.01232654]
 [-0.68170965 -0.29909426 -1.0102822  ...  2.8576348  -0.3050571
   0.34133095]
 [-0.8531192  -0.36630845 -0.62820077 ... -0.61596733  0.7249935
  -1.9096984 ]]
<NDArray 4096x4096 @cpu(0)>
B is  
[[-0.96239    -1.1772714   0.8881126  ... -2.4473324   2.36512
  -1.6756567 ]
 [-0.991634    1.9842013   0.48277834 ... -2.0245044   2.150161
  -0.3272309 ]
 [ 0.9540136  -1.7355976   1.0163084  ... -0.24425845  0.1050083
  -0.06981051]
 ...
 [ 0.818202   -1.262355   -0.6821951  ... -0.43549857  1.9019656
  -0.6036681 ]
 [ 1.1291006   0.14955084 -0.53832364 ...  1.4364871   0.3510555
  -0.33290246]
 [ 0.07341862  0.3190952  -1.7069333  ...  0.06805606 -0.35510552
   0

##### 5. Running these again in GPU we see that....its lightning fast!

## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

1. A symmetrix matrix $B \in \mathbb{S}^{n}$ is said to be positive semidefinite (PSD) if the associated quadratic form is non-negative, i.e.,
$$ x^TBx \geq 0,  \forall x \in \mathbb{R}^{n}. \\  \\
Hence, \ x^TBx\ =  \ x^TADA^Tx \ = \ y  D  y^T \ = \sum_{i=1}^{n} \lambda_{i} y_{i}^2 \geq 0\ \  (change \ of \ variable \ y = A^Tx) \ $$ 
 
2. $B$, a real symmetrix matrix has a complete set of orthogonal eigenvectors for which the corresponding eigenvalues are all real numbers, which prevents us from having to work with complex field. Furthermore, $B$, a PSD matrix, is convex, which is a useful property for optimization. Using $ADA^T$ is more helpful when operations like products of B is involved.


## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [73]:
A = nd.array(nd.random.normal(shape=(4096,4096)))
B = nd.array(nd.random.normal(shape=(4096,4096)))
# 1. Compute ∥ABi∥2 one at a time and assign its outcome to ci directly.
start = time.time()
c = np.zeros((4096,1))
for i in range(4096):
    c[i] = nd.norm(nd.dot(A, B[:,i])).asscalar()
end = time.time()
print("Method 1 takes ", end - start)

# 2. Use an intermediate storage d in NDArray for assignments and copy to NumPy at the end.
start = time.time()
c = np.zeros((4096,1))
d = nd.zeros((4096,1))
for i in range(4096):
    d[i] = nd.norm(nd.dot(A, B[:,i])).asscalar()
c = d.asnumpy()
end = time.time()
print("Method 2 takes ", end - start)

Method 1 takes  13.952924013137817
Method 2 takes  11.560874938964844


## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [54]:
A = nd.array(nd.random.normal(shape=(4096,4096)))
B = nd.array(nd.random.normal(shape=(4096,4096)))
C = nd.array(nd.random.normal(shape=(4096,4096)))
C += nd.dot(A,B)
print("C is ", C)

C is  
[[ 100.98398    101.1101      40.03901   ...  -28.16421    102.478966
    -9.316385 ]
 [ -50.589855    -8.312013   -29.966002  ...   67.39362    -83.28623
   -30.244991 ]
 [   4.8621874   -8.885946    30.35806   ...  -58.85867     88.89731
    51.633156 ]
 ...
 [  21.778372   -19.319244   148.01637   ...   11.807255   -63.0304
   102.84949  ]
 [  -3.0462499  -82.469734    -1.0283301 ... -103.1664     -86.86461
   -48.00691  ]
 [  31.830433   -33.972042    15.693489  ...   -1.2420454   44.58544
   -33.874435 ]]
<NDArray 4096x4096 @cpu(0)>


## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [49]:
x = np.linspace(-10,10,201)
x = nd.array(x).reshape(201,1)
y = np.linspace(1,20,20)
y = nd.array(y)
A = x ** y
print("A is ", A)

A is  
[[-1.0000000e+01  1.0000000e+02 -1.0000000e+03 ...  9.9999998e+17
  -1.0000000e+19  1.0000000e+20]
 [-9.8999996e+00  9.8009995e+01 -9.7029889e+02 ...  8.3451318e+17
  -8.2616803e+18  8.1790629e+19]
 [-9.8000002e+00  9.6040001e+01 -9.4119208e+02 ...  6.9513558e+17
  -6.8123289e+18  6.6760824e+19]
 ...
 [ 9.8000002e+00  9.6040001e+01  9.4119208e+02 ...  6.9513558e+17
   6.8123289e+18  6.6760824e+19]
 [ 9.8999996e+00  9.8009995e+01  9.7029889e+02 ...  8.3451318e+17
   8.2616803e+18  8.1790629e+19]
 [ 1.0000000e+01  1.0000000e+02  1.0000000e+03 ...  9.9999998e+17
   1.0000000e+19  1.0000000e+20]]
<NDArray 201x20 @cpu(0)>
